In [3]:
# modified 1/13/2025:
# 1. filter data between 2019 and 2023 
# 2. change table name from DSC_PLBI_DB.APP_HOME_PRD.HOME_PERSISTENCY_NOWCO --> DSC_PLBI_DB.APP_HOME_DEV.HOME_PERSISTENCY_NOWCO_2019_2023
# 3. Creating table in APP_HOME_PRD schema
# 4. Change "with open" path as per your local rsa_plbiap01dy.p8 file location


import snowflake.connector
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import serialization
from datetime import *
import datetime
import sys, getpass

with open("/etc/security/snowflake/rsa_plbiap01dy.p8", "rb") as key:
    p_key= serialization.load_pem_private_key(
        key.read(),
        password='snowflake'.encode(),
        backend=default_backend()
    )

pkb = p_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption())

ctx = snowflake.connector.connect(
    user='plbiap01dy',
    account='hfsg_prod.us-east-1.privatelink',
    private_key=pkb,
    warehouse='DSC_PLBI_PRD_MFG_WHS',
    database='DSC_PLBI_DB',
    schema='APP_HOME_PRD',
    role='plbiap01dy_prd_pii_role'
    )


cs = ctx.cursor()  

query1='''create
or replace table DSC_PLBI_DB.APP_HOME_PRD.KEVIN_PETER as
SELECT
    *,
    CASE
        WHEN ATR_PRE_45 = 1
        AND (
            CANCEL_TRANS_DT - TO_DATE(TO_CHAR(CANC_EFF_DT), 'YYYYMMDD')
        ) <= -45
        AND CANC_EFF_DT = POL_EFF_DT THEN 1
        ELSE NULL
    END AS CANC_PRE_45,
    CASE
        WHEN ATR_PRE_30 = 1
        AND (
            CANCEL_TRANS_DT - TO_DATE(TO_CHAR(CANC_EFF_DT), 'YYYYMMDD')
        ) <= -30
        AND CANC_EFF_DT = POL_EFF_DT THEN 1
        ELSE NULL
    END AS CANC_PRE_30,
    CASE
        WHEN ATR_PRE_15 = 1
        AND (
            CANCEL_TRANS_DT - TO_DATE(TO_CHAR(CANC_EFF_DT), 'YYYYMMDD')
        ) <= -15
        AND CANC_EFF_DT = POL_EFF_DT THEN 1
        ELSE NULL
    END AS CANC_PRE_15,
    CASE
        WHEN ATR_PRE = 1
        AND (
            CANCEL_TRANS_DT - TO_DATE(TO_CHAR(CANC_EFF_DT), 'YYYYMMDD')
        ) <= 0
        AND CANC_EFF_DT = POL_EFF_DT THEN 1
        ELSE NULL
    END AS CANC_PRE,
    CASE
        WHEN LAG(CANCEL_FLAG, 1) OVER (
            PARTITION BY POLICY_ID
            ORDER BY
                POL_EFF_DT
        ) = 1 THEN '1'
        WHEN LAG(CANCEL_FLAG, 1) OVER (
            PARTITION BY POLICY_ID
            ORDER BY
                POL_EFF_DT
        ) = 0 THEN '0'
        ELSE NULL
    END AS PRIOR_TERM_CANC_IND
FROM
    (
        SELECT
            AAAA.*,CASE
                WHEN AAAA.LATEST_CANCEL = 1 THEN 1
                WHEN AAAA.LATEST_CANCEL = 0
                AND AAAA.ATR_E_365 = 1 THEN 0
                WHEN AAAA.LATEST_CANCEL = 0
                AND AAAA.ATR_E_365 = 0 THEN (
                    CASE
                        WHEN AAAA.ATR_E_330 = 1 THEN 1 - NVL(AAAA.DEV_FCTR, 1)
                        WHEN AAAA.ATR_E_300 = 1 THEN 1 - NVL(AAAA.DEV_FCTR, 1)
                        WHEN AAAA.ATR_E_270 = 1 THEN 1 - NVL(AAAA.DEV_FCTR, 1)
                        WHEN AAAA.ATR_E_240 = 1 THEN 1 - NVL(AAAA.DEV_FCTR, 1)
                        WHEN AAAA.ATR_E_210 = 1 THEN 1 - NVL(AAAA.DEV_FCTR, 1)
                        WHEN AAAA.ATR_E_180 = 1 THEN 1 - NVL(AAAA.DEV_FCTR, 1)
                        WHEN AAAA.ATR_E_150 = 1 THEN 1 - NVL(AAAA.DEV_FCTR, 1)
                        WHEN AAAA.ATR_E_120 = 1 THEN 1 - NVL(AAAA.DEV_FCTR, 1)
                        WHEN AAAA.ATR_E_90 = 1 THEN 1 - NVL(AAAA.DEV_FCTR, 1)
                        WHEN AAAA.ATR_E_60 = 1 THEN 1 - NVL(AAAA.DEV_FCTR, 1)
                        WHEN AAAA.ATR_E_30 = 1 THEN 1 - NVL(AAAA.DEV_FCTR, 1)
                        WHEN AAAA.ATR_E_FLAT = 1 THEN 1 - NVL(AAAA.DEV_FCTR, 1)
                        ELSE NULL
                    END
                )
                ELSE NULL
            END AS ULT_CANC_CNT,
            CASE
                WHEN (
                    AS_OF_DATE - TO_DATE(TO_CHAR(POL_EFF_DT), 'yyyymmdd')
                ) >= -45 THEN 1
                ELSE NULL
            END AS ATR_PRE_45,
            CASE
                WHEN (
                    AS_OF_DATE - TO_DATE(TO_CHAR(POL_EFF_DT), 'yyyymmdd')
                ) >= -30 THEN 1
                ELSE NULL
            END AS ATR_PRE_30,
            CASE
                WHEN (
                    AS_OF_DATE - TO_DATE(TO_CHAR(POL_EFF_DT), 'yyyymmdd')
                ) >= -15 THEN 1
                ELSE NULL
            END AS ATR_PRE_15,
            CASE
                WHEN (
                    AS_OF_DATE - TO_DATE(TO_CHAR(POL_EFF_DT), 'yyyymmdd')
                ) >= 0 THEN 1
                ELSE NULL
            END AS ATR_PRE
        FROM
            (
                SELECT
                    AAA.*,
                    CCC.POL_EFF_YRMO,
                    CCC.POLYR,
                    CCC.CURR_YE_OUTLK_PCT,
                    CCC.SNR,
                    CCC.SNR_PREMIUM_CHANGE,
                    CCC.OVERALL_EXPECTED_RATE,
                    CCC.EXPECTED_PREMIUM_CHANGE,
                    CCC.FRIAR_PREMIUM_CHANGE,
                    CCC.LOW_PREM_FLAG,
                    CCC.OVERALL_EXPECTED_RATE_GRP,
                    GREATEST(
                        AAA.CANCEL_E_365,
                        AAA.CANCEL_E_330,
                        AAA.CANCEL_E_300,
                        AAA.CANCEL_E_270,
                        AAA.CANCEL_E_240,
                        AAA.CANCEL_E_210,
                        AAA.CANCEL_E_180,
                        AAA.CANCEL_E_150,
                        AAA.CANCEL_E_120,
                        AAA.CANCEL_E_90,
                        AAA.CANCEL_E_60,
                        AAA.CANCEL_E_30,
                        AAA.CANCEL_E_FLAT
                    ) AS LATEST_CANCEL,CASE
                        WHEN AAA.PERSISTENCY_PERIOD = '1' THEN 1
                        ELSE NVL(BBB.DEV_FCTR, 1)
                    END AS DEV_FCTR,
                    AAA.ATR_E_FLAT AS ULT_ATR_CNT,
                    LAG(AAA.ACCT_CR_IND) OVER (
                        PARTITION BY AAA.POLICY_ID
                        ORDER BY
                            AAA.POL_EFF_DT
                    ) PRIOR_TERM_ACCT_CR_IND
                FROM
                    (
                        SELECT
                            AA.*
                        FROM
                            (
                                SELECT
                                    B.AS_OF_DATE,
                                    A.AGMT_GID,
                                    A.POL_PRD_ID,
                                    B.POL_SYM_CD,
                                    B.PROD_RO_CD,
                                    B.POL_NUM,
                                    B.POLICY_ID,
                                    B.POL_EFF_DT,
                                    B.POL_EXP_DT,
                                    B.POL_TERM_LATST_TRANS_TMSP,
                                    B.POL_STATUS_CD,
                                    1 POL_CNT,CASE
                                        WHEN B.POL_STATUS_CD = 2 THEN 1
                                        ELSE 0
                                    END FLAT_CANCEL_FLAG,
                                    NULLIFZERO(B.CANC_EFF_DT) CANC_EFF_DT,
                                    B.REWRITE_IND,
                                    B.CANC_RW_IND_DESC,
                                    B.CANC_RW_ID,
                                    B.CANC_RW_ORDER_ID,
                                    B.POL_FORM_CD,
                                    B.FORM,
                                    B.RATE_PLAN_CD,
                                    B.BUS_UNIT_ABBR,
                                    B.RISK_STATE,
                                    B.POL_NEW_RNW_CD,
                                    B.RATE_TIER_INS_SCORE_CD,
                                    B.RATE_TIER_AARP_MEMB_DAY_CNT,
                                    B.RATE_TIER_AARP_MEMB_STATUS_CD,
                                    B.SCORE,
                                    B.ACCOUNT_CREDIT,
                                    B.HOME_QUALITY_PERSONA,
                                    B.COVERAGE_A,
                                    B.COV_A_LMT,
                                    B.COVERAGE_C,
                                    B.COV_C_LMT,
                                    B.COVERAGE_E,
                                    B.COV_E_LMT,
                                    B.ROOF_AGE,
                                    B.ROOF_AGE_GROUP,
                                    B.ROOF_UPD_YR,
                                    B.ROOF_TYPE,
                                    B.INS_SCORE_GRP,
                                    B.YEAR_BUILT,
                                    B.NEW_BUS_TIER_IND,
                                    B.PRIOR_CAT_CLM_CNT,
                                    B.PRIOR_FIRE_CLM_CNT,
                                    B.PRIOR_HAIL_CLM_CNT,
                                    B.PRIOR_LIAB_CLM_CNT,
                                    B.PRIOR_LIGHT_CLM_CNT,
                                    B.PRIOR_OTH_CLM_CNT,
                                    B.PRIOR_THEFT_CLM_CNT,
                                    B.PRIOR_WIND_CLM_CNT,
                                    B.PRIOR_WTR_CLM_CNT,
                                    B.DWELLING_AGE_GROUP,
                                    B.YEARS_WITH_HIG,
                                    B.DWELLING_LOC_ZIP_CD,
                                    B.DWELLING_LOC_PLUS_FOUR_ZIP_CD,
                                    B.FIRE_PREM,
                                    B.LIGHTNG_PREM,
                                    B.LIAB_PREM,
                                    B.THEFT_PREM,
                                    B.WAT_NW_PREM,
                                    B.WAT_W_PREM,
                                    B.WIND_PREM,
                                    B.HAIL_PREM,
                                    B.OTH_PREM,
                                    B.HURCANE_PREM,
                                    B.WILDFIRE_PREM,
                                    B.FIRE_F_EQ_PREM,
                                    B.ALL_PERIL_PREM,
                                    B.DWEL_LOC_QLTY_CD,
                                    NVL(
                                        LAG(B.TERM_WRIT_PREM, 1) OVER (
                                            PARTITION BY POLICY_ID
                                            ORDER BY
                                                B.POL_EFF_DT
                                        ),
                                        nvl(C.PREV_POL_ID_LATST_PREM, 0)
                                    ) PREV_TERM_LATEST_PREM_AMT,
                                    NVL(
                                        LAG(B.TERM_WRIT_PREM, 2) OVER (
                                            PARTITION BY POLICY_ID
                                            ORDER BY
                                                B.POL_EFF_DT
                                        ),
                                        0
                                    ) PREV_TWO_TERM_LATEST_PREM_AMT,
                                    B.TERM_WRIT_PREM,
                                    B.PROTECTION_CLASS_PT1_CD,
                                    B.RATE_TIER_HOME_BASIC_DED_AMT,
                                    D.FRST_TRANS_PREM,
                                    B.PREV_TRM_FRST_TRANS_PREM,CASE
                                        WHEN B.CANC_EFF_DT = 0
                                        OR B.CANC_EFF_DT IS NULL THEN -1
                                        ELSE TO_DATE (TO_CHAR(B.CANC_EFF_DT), 'yyyymmdd') - TO_DATE (TO_CHAR(B.POL_EFF_DT), 'yyyymmdd')
                                    END AS NUM_DAYS_CANC,
                                    TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') NUM_DAYS_EFF,CASE
                                        WHEN B.CANC_EFF_DT IS NULL
                                        OR B.CANC_EFF_DT = 0
                                        OR TO_DATE(TO_CHAR(B.CANC_EFF_DT), 'yyyymmdd') - TO_DATE (TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') = 365 THEN 0
                                        ELSE 1
                                    END AS CANCEL_FLAG,CASE
                                        WHEN B.CANC_EFF_DT > 0
                                        AND B.CANC_EFF_DT IS NOT NULL
                                        AND (
                                            TO_DATE(TO_CHAR(B.CANC_EFF_DT), 'YYYYMMDD') - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'YYYYMMDD') <= 0
                                        )
                                        AND (
                                            TO_DATE(B.AS_OF_DATE) >= TO_DATE(TO_CHAR(B.POL_EFF_DT), 'YYYYMMDD')
                                        ) THEN 1
                                        ELSE 0
                                    END AS CANCEL_E_FLAT,CASE
                                        WHEN CANC_EFF_DT > 0
                                        AND CANC_EFF_DT IS NOT NULL
                                        AND TO_DATE(TO_CHAR(B.CANC_EFF_DT), 'yyyymmdd') - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') <= 30
                                        AND TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 30 THEN 1
                                        ELSE 0
                                    END AS CANCEL_E_30,CASE
                                        WHEN CANC_EFF_DT > 0
                                        AND CANC_EFF_DT IS NOT NULL
                                        AND TO_DATE(TO_CHAR(B.CANC_EFF_DT), 'yyyymmdd') - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') <= 60
                                        AND TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 60 THEN 1
                                        ELSE 0
                                    END AS CANCEL_E_60,CASE
                                        WHEN CANC_EFF_DT > 0
                                        AND CANC_EFF_DT IS NOT NULL
                                        AND TO_DATE(TO_CHAR(B.CANC_EFF_DT), 'yyyymmdd') - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') <= 90
                                        AND TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 90 THEN 1
                                        ELSE 0
                                    END AS CANCEL_E_90,CASE
                                        WHEN CANC_EFF_DT > 0
                                        AND CANC_EFF_DT IS NOT NULL
                                        AND TO_DATE(TO_CHAR(B.CANC_EFF_DT), 'yyyymmdd') - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') <= 120
                                        AND TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 120 THEN 1
                                        ELSE 0
                                    END AS CANCEL_E_120,CASE
                                        WHEN CANC_EFF_DT > 0
                                        AND CANC_EFF_DT IS NOT NULL
                                        AND TO_DATE(TO_CHAR(B.CANC_EFF_DT), 'yyyymmdd') - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') <= 150
                                        AND TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 150 THEN 1
                                        ELSE 0
                                    END AS CANCEL_E_150,CASE
                                        WHEN CANC_EFF_DT > 0
                                        AND CANC_EFF_DT IS NOT NULL
                                        AND TO_DATE(TO_CHAR(B.CANC_EFF_DT), 'yyyymmdd') - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') <= 180
                                        AND TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 180 THEN 1
                                        ELSE 0
                                    END AS CANCEL_E_180,CASE
                                        WHEN CANC_EFF_DT > 0
                                        AND CANC_EFF_DT IS NOT NULL
                                        AND TO_DATE(TO_CHAR(B.CANC_EFF_DT), 'yyyymmdd') - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') <= 210
                                        AND TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 210 THEN 1
                                        ELSE 0
                                    END AS CANCEL_E_210,CASE
                                        WHEN CANC_EFF_DT > 0
                                        AND CANC_EFF_DT IS NOT NULL
                                        AND TO_DATE(TO_CHAR(B.CANC_EFF_DT), 'yyyymmdd') - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') <= 240
                                        AND TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 240 THEN 1
                                        ELSE 0
                                    END AS CANCEL_E_240,CASE
                                        WHEN CANC_EFF_DT > 0
                                        AND CANC_EFF_DT IS NOT NULL
                                        AND TO_DATE(TO_CHAR(B.CANC_EFF_DT), 'yyyymmdd') - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') <= 270
                                        AND TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 270 THEN 1
                                        ELSE 0
                                    END AS CANCEL_E_270,CASE
                                        WHEN CANC_EFF_DT > 0
                                        AND CANC_EFF_DT IS NOT NULL
                                        AND TO_DATE(TO_CHAR(B.CANC_EFF_DT), 'yyyymmdd') - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') <= 300
                                        AND TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 300 THEN 1
                                        ELSE 0
                                    END AS CANCEL_E_300,CASE
                                        WHEN CANC_EFF_DT > 0
                                        AND CANC_EFF_DT IS NOT NULL
                                        AND TO_DATE(TO_CHAR(B.CANC_EFF_DT), 'yyyymmdd') - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') <= 330
                                        AND TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 330 THEN 1
                                        ELSE 0
                                    END AS CANCEL_E_330,CASE
                                        WHEN CANC_EFF_DT > 0
                                        AND CANC_EFF_DT IS NOT NULL
                                        AND TO_DATE(TO_CHAR(B.CANC_EFF_DT), 'yyyymmdd') - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') < 365
                                        AND TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 365 THEN 1
                                        ELSE 0
                                    END AS CANCEL_E_365,CASE
                                        WHEN TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 0 THEN 1
                                        ELSE 0
                                    END AS ATR_E_FLAT,CASE
                                        WHEN TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 30 THEN 1
                                        ELSE 0
                                    END AS ATR_E_30,CASE
                                        WHEN TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 60 THEN 1
                                        ELSE 0
                                    END AS ATR_E_60,CASE
                                        WHEN TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 90 THEN 1
                                        ELSE 0
                                    END AS ATR_E_90,CASE
                                        WHEN TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 120 THEN 1
                                        ELSE 0
                                    END AS ATR_E_120,CASE
                                        WHEN TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 150 THEN 1
                                        ELSE 0
                                    END AS ATR_E_150,CASE
                                        WHEN TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 180 THEN 1
                                        ELSE 0
                                    END AS ATR_E_180,CASE
                                        WHEN TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 210 THEN 1
                                        ELSE 0
                                    END AS ATR_E_210,CASE
                                        WHEN TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 240 THEN 1
                                        ELSE 0
                                    END AS ATR_E_240,CASE
                                        WHEN TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 270 THEN 1
                                        ELSE 0
                                    END AS ATR_E_270,CASE
                                        WHEN TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 300 THEN 1
                                        ELSE 0
                                    END AS ATR_E_300,CASE
                                        WHEN TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 330 THEN 1
                                        ELSE 0
                                    END AS ATR_E_330,CASE
                                        WHEN TO_DATE(B.AS_OF_DATE) - TO_DATE(TO_CHAR(B.POL_EFF_DT), 'yyyymmdd') >= 365 THEN 1
                                        ELSE 0
                                    END AS ATR_E_365,CASE
                                        WHEN B.POL_NEW_RNW_CD = 'N'
                                        OR PREV_TERM_LATEST_PREM_AMT = 0 THEN NULL
                                        ELSE (
                                            (B.FRST_TRANS_PREM / PREV_TERM_LATEST_PREM_AMT) - 1
                                        )
                                    END RENEWAL_PREMIUM_CHANGE,CASE
                                        WHEN B.POL_NEW_RNW_CD = 'N'
                                        OR PREV_TWO_TERM_LATEST_PREM_AMT = 0 THEN NULL
                                        ELSE (
                                            (
                                                B.FRST_TRANS_PREM / PREV_TWO_TERM_LATEST_PREM_AMT
                                            ) - 1
                                        )
                                    END RENEWAL_PREMIUM_CHANGE_TWO_TERM,CASE
                                        WHEN B.POL_NEW_RNW_CD = 'N'
                                        OR PREV_TERM_LATEST_PREM_AMT = 0 THEN NULL
                                        WHEN (
                                            (B.FRST_TRANS_PREM / PREV_TERM_LATEST_PREM_AMT) - 1
                                        ) < -0.05 THEN 'a. < -5'
                                        WHEN (
                                            (B.FRST_TRANS_PREM / PREV_TERM_LATEST_PREM_AMT) - 1
                                        ) >= -0.05
                                        AND (
                                            (B.FRST_TRANS_PREM / PREV_TERM_LATEST_PREM_AMT) - 1
                                        ) < 0 THEN 'b. -5 to 0'
                                        WHEN (
                                            (B.FRST_TRANS_PREM / PREV_TERM_LATEST_PREM_AMT) - 1
                                        ) >= 0
                                        AND (
                                            (B.FRST_TRANS_PREM / PREV_TERM_LATEST_PREM_AMT) - 1
                                        ) < 0.1 THEN 'c. 0 to 9'
                                        WHEN (
                                            (B.FRST_TRANS_PREM / PREV_TERM_LATEST_PREM_AMT) - 1
                                        ) >= 0.1
                                        AND (
                                            (B.FRST_TRANS_PREM / PREV_TERM_LATEST_PREM_AMT) - 1
                                        ) < 0.2 THEN 'd. 10 to 19'
                                        WHEN (
                                            (B.FRST_TRANS_PREM / PREV_TERM_LATEST_PREM_AMT) - 1
                                        ) >= 0.2 THEN 'e. > 20'
                                    END AS RENEWAL_PREMIUM_CHANGE_GRP,
                                    B.MARKET_CHANNEL,
                                    B.AARP_THRU_AGNT_IND,
                                    B.PLCY_CANCEL_REASON,CASE
                                        WHEN TO_DATE (B.AS_OF_DATE) - TO_DATE (to_char(B.POL_EFF_DT), 'YYYYMMDD') >= 365 THEN '1'
                                        WHEN TO_DATE (B.AS_OF_DATE) - TO_DATE (to_char(B.POL_EFF_DT), 'YYYYMMDD') >= 185
                                        AND B.POL_TERM_MO_CNT = 6 THEN '1'
                                        WHEN TO_DATE (B.AS_OF_DATE) - TO_DATE (to_char(B.POL_EFF_DT), 'YYYYMMDD') >= 330 THEN '330-Ult'
                                        WHEN TO_DATE (B.AS_OF_DATE) - TO_DATE (to_char(B.POL_EFF_DT), 'YYYYMMDD') >= 300 THEN '300-Ult'
                                        WHEN TO_DATE (B.AS_OF_DATE) - TO_DATE (to_char(B.POL_EFF_DT), 'YYYYMMDD') >= 270 THEN '270-Ult'
                                        WHEN TO_DATE (B.AS_OF_DATE) - TO_DATE (to_char(B.POL_EFF_DT), 'YYYYMMDD') >= 240 THEN '240-Ult'
                                        WHEN TO_DATE (B.AS_OF_DATE) - TO_DATE (to_char(B.POL_EFF_DT), 'YYYYMMDD') >= 210 THEN '210-Ult'
                                        WHEN TO_DATE (B.AS_OF_DATE) - TO_DATE (to_char(B.POL_EFF_DT), 'YYYYMMDD') >= 180 THEN '180-Ult'
                                        WHEN TO_DATE (B.AS_OF_DATE) - TO_DATE (to_char(B.POL_EFF_DT), 'YYYYMMDD') >= 150 THEN '150-Ult'
                                        WHEN TO_DATE (B.AS_OF_DATE) - TO_DATE (to_char(B.POL_EFF_DT), 'YYYYMMDD') >= 120 THEN '120-Ult'
                                        WHEN TO_DATE (B.AS_OF_DATE) - TO_DATE (to_char(B.POL_EFF_DT), 'YYYYMMDD') >= 90 THEN '90-Ult'
                                        WHEN TO_DATE (B.AS_OF_DATE) - TO_DATE (to_char(B.POL_EFF_DT), 'YYYYMMDD') >= 60 THEN '60-Ult'
                                        WHEN TO_DATE (B.AS_OF_DATE) - TO_DATE (to_char(B.POL_EFF_DT), 'YYYYMMDD') >= 30 THEN '30-Ult'
                                        WHEN TO_DATE (B.AS_OF_DATE) >= TO_DATE (to_char(B.POL_EFF_DT), 'YYYYMMDD') THEN '0-Ult'
                                        ELSE NULL
                                    END AS PERSISTENCY_PERIOD,
                                    B.RATE_TBL_DT,
                                    LAG(B.RATE_TBL_DT, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            B.POL_EFF_DT
                                    ) PREV_TERM_RATE_TBL_DT,CASE
                                        WHEN B.RATE_TBL_DT <> PREV_TERM_RATE_TBL_DT THEN 1
                                        ELSE 0
                                    END AS RATE_TBL_DT_CHNG_IND,
                                    LAG(B.COVERAGE_A, 1) OVER (
                                        PARTITION BY POLICY_ID
                                        ORDER BY
                                            B.POL_EFF_DT
                                    ) PREV_TERM_COV_A_RNG,
                                    B.ACCOUNT_ID,
                                    B.FRST_CHRG_LOSS_CNT,
                                    B.FRST_INS_SCORE_CD,
                                    B.FRST_INS_SCORE_MODEL_TYPE_CD,
                                    B.FRST_RATING_TERR_CD,
                                    B.FRST_PROTECTION_CLASS_PT1_CD,
                                    B.FRST_ITV_PCT,
                                    LAG(B.LATST_CHRG_LOSS_CNT, 1) OVER (
                                        PARTITION BY POLICY_ID
                                        ORDER BY
                                            B.POL_EFF_DT
                                    ) PREV_TERM_LOSS_CNT,
                                    LAG(B.LATST_INS_SCORE_CD, 1) OVER (
                                        PARTITION BY POLICY_ID
                                        ORDER BY
                                            B.POL_EFF_DT
                                    ) PREV_TERM_INS_SCORE_CD,
                                    LAG(B.LATST_INS_SCORE_MODEL_TYPE_CD, 1) OVER (
                                        PARTITION BY POLICY_ID
                                        ORDER BY
                                            B.POL_EFF_DT
                                    ) PREV_TERM_INS_SCORE_MODEL_TYPE_CD,
                                    LAG(B.LATST_RATING_TERR_CD, 1) OVER (
                                        PARTITION BY POLICY_ID
                                        ORDER BY
                                            B.POL_EFF_DT
                                    ) PREV_TERM_RATING_TERR_CD,
                                    LAG(B.LATST_PROTECTION_CLASS_PT1_CD, 1) OVER (
                                        PARTITION BY POLICY_ID
                                        ORDER BY
                                            B.POL_EFF_DT
                                    ) PREV_TERM_PROTECTION_CLASS_PT1_CD,
                                    LAG(B.LATST_ITV_PCT, 1) OVER (
                                        PARTITION BY POLICY_ID
                                        ORDER BY
                                            B.POL_EFF_DT
                                    ) PREV_TERM_ITV_PCT,CASE
                                        WHEN PREV_TERM_LOSS_CNT = FRST_CHRG_LOSS_CNT THEN 1
                                        ELSE 0
                                    END CHNG_LOSS_IND,CASE
                                        WHEN PREV_TERM_INS_SCORE_CD = FRST_INS_SCORE_CD THEN 1
                                        ELSE 0
                                    END CHNG_INS_SCORE_IND,CASE
                                        WHEN PREV_TERM_INS_SCORE_MODEL_TYPE_CD = FRST_INS_SCORE_MODEL_TYPE_CD THEN 1
                                        ELSE 0
                                    END CHNG_INS_SCORE_MODEL_TYPE_IND,CASE
                                        WHEN PREV_TERM_RATING_TERR_CD = FRST_RATING_TERR_CD THEN 1
                                        ELSE 0
                                    END CHNG_RATING_TERR_IND,CASE
                                        WHEN PREV_TERM_PROTECTION_CLASS_PT1_CD = FRST_PROTECTION_CLASS_PT1_CD THEN 1
                                        ELSE 0
                                    END CHNG_PROTECTION_CLASS_PT1_IND,CASE
                                        WHEN PREV_TERM_ITV_PCT = FRST_ITV_PCT THEN 1
                                        ELSE 0
                                    END CHNG_ITV_IND,
                                    B.ORIG_POL_EFF_DT,CASE
                                        WHEN B.ACCT_CR_IND = 1 THEN 'Y'
                                        WHEN B.ACCT_CR_IND = 2 THEN 'N'
                                        ELSE NULL
                                    END AS ACCT_CR_IND,
                                    B.PRR_CARR_DESC,
                                    B.PCARR_DESC,
                                    B.WRITING_COMPANY,
                                    B.DERIV_INSURED_AGE,
                                    B.POL_DED_AMT,
                                    B.BASIC_POL_PREM,CASE
                                        WHEN B.POL_TERM_MO_CNT = 6 THEN B.TERM_WRIT_PREM * 2
                                        ELSE B.TERM_WRIT_PREM
                                    END POL_ANNL_TTL_BILL_PREM_AMT,
                                    B.XINST_MARITAL_STAT_HOUSEHOLD,
                                    B.POLFORM_PAYER,
                                    B.AARP_MEMBER_STATUS_CD,
                                    B.AARP_MEMB_STATUS_DESC_AUTO,
                                    B.AVG_PRM_GRP2_AUTO,
                                    B.CURRENT_TERM_AUTO_PREM,
                                    LAG(B.CURRENT_TERM_AUTO_PREM, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            B.POL_EFF_DT
                                    ) PREV_TERM_AUTO_PREM,
                                    CASE
                                        WHEN B.CURRENT_TERM_AUTO_PREM IS NOT NULL
                                        AND (
                                            LAG(B.CURRENT_TERM_AUTO_PREM, 1) OVER(
                                                PARTITION BY B.POLICY_ID
                                                ORDER BY
                                                    B.POL_EFF_DT
                                            )
                                        ) IS NOT NULL
                                        AND (
                                            LAG(B.TERM_WRIT_PREM, 1) OVER (
                                                PARTITION BY POLICY_ID
                                                ORDER BY
                                                    B.POL_EFF_DT
                                            )
                                        ) IS NOT NULL THEN DIV0(
                                            (B.TERM_WRIT_PREM + B.CURRENT_TERM_AUTO_PREM),
                                            (
                                                LAG(B.TERM_WRIT_PREM, 1) OVER (
                                                    PARTITION BY POLICY_ID
                                                    ORDER BY
                                                        B.POL_EFF_DT
                                                ) + (
                                                    LAG(B.CURRENT_TERM_AUTO_PREM, 1) OVER(
                                                        PARTITION BY B.POLICY_ID
                                                        ORDER BY
                                                            B.POL_EFF_DT
                                                    )
                                                )
                                            )
                                        ) - 1
                                        ELSE NULL
                                    END AS PREM_CHNG_COMBINED,
                                    B.COVERAGE_B,
                                    B.COV_B_LMT,
                                    B.COV_D_LMT,
                                    B.COV_F_LMT,
                                    B.ITV_PCT,
                                    B.NM_INSURED_NM,
                                    B.INSRD_MAIL_STREET_ADDR,
                                    B.ACTIVE_MODEL_SCR,
                                    B.FLAT_MODEL_SCR,
                                    B.MID_MODEL_SCR,
                                    B.PREV_POL_ID,
                                    B.CANC_TERM_LATST_PREM_AMT,CASE
                                        WHEN B.POLICY_ID != B.PREV_POL_ID THEN NVL(C.PREV_POL_ID_LATST_PREM, 0)
                                        Else '0'
                                    END PREV_POL_ID_LATST_PREM,
                                    CASE
                                        WHEN B.CANC_EFF_DT != 0 THEN TO_DATE(TO_CHAR(NULLIFZERO(B.POL_ENTRY_DT)), 'YYYYMMDD')
                                        ELSE NULL
                                    END CANCEL_TRANS_DT,
                                    B.FIRELINE_SCORE_CD,
                                    B.DOG_ON_PRM_IND,
                                    B.DIST_TO_COAST_FEET_CNT,
                                    B.SWIMMING_POOL_IND,
                                    B.CONSTRUCTION_MTRL_DESC,
                                    B.DIST_TO_COAST_GRP,
                                    B.SQ_FT_GRP,
                                    B.COV_C_GRP_TC,
                                    B.AARP_MEMBERSHIP_DAY_CNT_GRP,
                                    B.AARP_MEMBERSHIP_DAY_CNT,
                                    B.AARP_MEMBER_IND,
                                    B.GIA_IND,
                                    B.TOTAL_AREA_SQUARE_FEET_CNT,
                                    B.DWELLING_CONSTRUCTION_YR,
                                    B.PROD_AGNCY_ID,
                                    B.PRODUCER_NAME,
                                    B.MASTER_AGENCY_CODE,
                                    B.MASTER_AGENCY_NAME,
                                    B.AFF_PRTNR_SUB_GROUP_CD,
                                    B.ORIG_POL_YR,
                                    B.LOCATION_ID,
                                    B.LOCATION_NAME,
                                    B.AGNCY_BRKR_AGNCY_CLU_CD,
                                    B.LOCATION_CITY,
                                    B.LOCATION_STATE,
                                    B.SALES_REP_NAME,
                                    B.SALES_MANAGER_NAME,
                                    B.INS_SCR_DECILE_QUOTE_ST,
                                    B.INS_SCR_DECILE_QUOTE_CW,
                                    B.INS_SCR_DECILE_INFORCE_CW,
                                    B.MKTG_CAMPGN_CD,
                                    B.GRID_ID_500M,
                                    B.COVERAGE_D,
                                    B.MEDIA_GROUP_CD_DESC,
                                    B.BILL_PYMENT_FREQ_DESC,
                                    B.BILL_PYMNT_METH_DESC,
                                    B.SECND_HEAT_SYS_TYP_DESC,
                                    B.HH_MEMB_CNT,
                                    B.MANUAL_POLICY_IND,
                                    B.HURR_COV_DEDUC_TYP_DESC,
                                    B.HURR_COV_DEDUC_DESC,
                                    B.HURR_COV_DEDUC_CD,
                                    B.MORTGAGEE_CNT,
                                    B.INS_SCORE_CD,
                                    B.CALCULATED_AVG_NM_INSURED_AGE AS CALC_AVG_NM_INSD_AGE,
                                    B.ROOF_COMPL_YR,
                                    B.TRANS_TYP_DESC,
                                    TO_NUMBER(B.YEARS_WITH_HIG, 10, 1) AS HTFD_POL_YR_CNT,
                                    CASE
                                        WHEN B.DWELLING_LOC_PLUS_FOUR_ZIP_CD IS NOT NULL THEN B.DWELLING_LOC_ZIP_CD || '-' || B.DWELLING_LOC_PLUS_FOUR_ZIP_CD
                                        WHEN B.DWELLING_LOC_PLUS_FOUR_ZIP_CD IS NULL THEN B.DWELLING_LOC_ZIP_CD
                                        ELSE NULL
                                    END DWELL_LOC_ADDR_FULL_ZIP_CD,
                                    B.PRTCT_CLASS_CD,
                                    B.MRKT_MEDIA_GRP_DESCR,
                                    B.WRIT_CO_CD,
                                    B.DWELLING_SUBSTATE,
                                    B.INS_SCR_DECILE_INFORCE_ST,
                                    B.EQ_GRSS_AAL_AMT_LATEST,
                                    B.FFE_GRSS_AAL_AMT_LATEST,
                                    B.HURR_NEAR_TERM_GRSS_AAL_AMT_LATEST,
                                    B.HURR_LT_GRSS_AAL_AMT_LATEST,
                                    B.TOR_GRSS_AAL_AMT_LATEST,
                                    B.HAIL_GRSS_AAL_AMT_LATEST,
                                    B.WLDFR_GRSS_AAL_AMT_LATEST,
                                    B.RMS_WT_GR_AAL_AMT_LATEST,
                                    B.TOT_AAL_LATEST,
                                    B.AAL_YRMO_DT_LATEST,
                                    B.AAL_DATA_IND_LATEST,
                                    B.TOTAL_AAL_CAT_LOAD,
                                    B.TOT_LATEST_AAL_CAT_LOAD_WP_GRP,
                                    LAG(B.EQ_GRSS_AAL_AMT_LATEST, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            POL_EFF_DT
                                    ) PRR_EQ_GRSS_AAL_AMT_LATEST,
                                    LAG(B.FFE_GRSS_AAL_AMT_LATEST, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            POL_EFF_DT
                                    ) PRR_FFE_GRSS_AAL_AMT_LATEST,
                                    LAG(B.HURR_NEAR_TERM_GRSS_AAL_AMT_LATEST, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            POL_EFF_DT
                                    ) PRR_HURR_NEAR_TERM_GRSS_AAL_AMT_LATEST,
                                    LAG(B.HURR_LT_GRSS_AAL_AMT_LATEST, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            POL_EFF_DT
                                    ) PRR_HURR_LT_GRSS_AAL_AMT_LATEST,
                                    LAG(B.TOR_GRSS_AAL_AMT_LATEST, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            POL_EFF_DT
                                    ) PRR_TOR_GRSS_AAL_AMT_LATEST,
                                    LAG(B.HAIL_GRSS_AAL_AMT_LATEST, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            POL_EFF_DT
                                    ) PRR_HAIL_GRSS_AAL_AMT_LATEST,
                                    LAG(B.WLDFR_GRSS_AAL_AMT_LATEST, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            POL_EFF_DT
                                    ) PRR_WLDFR_GRSS_AAL_AMT_LATEST,
                                    LAG(B.RMS_WT_GR_AAL_AMT_LATEST, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            POL_EFF_DT
                                    ) PRR_RMS_WT_GR_AAL_AMT_LATEST,
                                    LAG(B.TOT_AAL_LATEST, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            POL_EFF_DT
                                    ) PRR_TOT_AAL_LATEST,
                                    LAG(B.AAL_YRMO_DT_LATEST, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            POL_EFF_DT
                                    ) PRR_AAL_YRMO_DT_LATEST,
                                    LAG(B.AAL_DATA_IND_LATEST, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            POL_EFF_DT
                                    ) PRR_AAL_DATA_IND_LATEST,
                                    LAG(B.TOTAL_AAL_CAT_LOAD, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            POL_EFF_DT
                                    ) PRR_TOTAL_AAL_CAT_LOAD,
                                    LAG(B.TOT_LATEST_AAL_CAT_LOAD_WP_GRP, 1) OVER(
                                        PARTITION BY B.POLICY_ID
                                        ORDER BY
                                            POL_EFF_DT
                                    ) PRR_TOT_LATEST_AAL_CAT_LOAD_WP_GRP,
                                    CASE
                                        WHEN B.AAL_DATA_IND_LATEST = 'N' THEN 'NO AAL'
                                        WHEN B.TOTAL_AAL_CAT_LOAD IS NULL
                                        OR PRR_TOTAL_AAL_CAT_LOAD IS NULL THEN 'UNKNOWN'
                                        WHEN B.TOTAL_AAL_CAT_LOAD = PRR_TOTAL_AAL_CAT_LOAD THEN 'NO CHANGE'
                                        WHEN B.TOTAL_AAL_CAT_LOAD > PRR_TOTAL_AAL_CAT_LOAD THEN 'INCREASE'
                                        WHEN B.TOTAL_AAL_CAT_LOAD < PRR_TOTAL_AAL_CAT_LOAD THEN 'DECREASE'
                                    END AS AAL_CAT_LOAD_CHANGE,
                                    B.STATE_PRODUCT_REGION,
                                    B.DWELL_STRY_CNT_CD,
                                    B.POL_PACKAGE,
                                    B.WND_HAIL_DEDUC_AMT,
                                    B.LBC_IND,
                                    B.RATE_RULE_EFF_DT,
                                    B.ROOF_ARCHTCT_GEOM_TYP_DESC,
                                    B.ORIG_QTE_CNTCT_METH_CD,
                                    B.MKT_CHNNL_NM,
                                    B.MKTG_MEDIA_GRP_DESC,
                                    B.MKTG_MEDIA_DESC
                                FROM
                                    PRD_PL_DB.APP_SCDI_DM.PL_HOME_POLICY_SNAPSHOT_FACT_VW A,
                                    /*( SELECT A.AGMT_GID, MAX(A.POL_PRD_ID) POL_PRD_ID FROM HOME_POLICY_CHAR_REF A GROUP BY A.AGMT_GID ) A,*/
                                    HOME_POLICY_CHAR_REF B,(
                                        SELECT
                                            AA.AGMT_PK_ID,
                                            AA.AGMT_GID,
                                            AA.MASTER_AGMT_GID,
                                            AA.SRCE_EFF_START_TMSP TERM_TRANS_FRST_DT,
                                            AA.TERM_WRIT_PREM FRST_TRANS_PREM,
                                            NVL (
                                                LAG (AA.TERM_WRIT_PREM, 1) OVER (
                                                    PARTITION BY AA.POLICY_ID
                                                    ORDER BY
                                                        TO_DATE(to_char(AA.POL_EFF_DT_PK_ID), 'YYYYMMDD')
                                                ),
                                                0
                                            ) PREV_TRM_FRST_TRANS_PREM
                                        FROM
                                            (
                                                SELECT
                                                    B.AGMT_PK_ID,
                                                    B.AGMT_GID,
                                                    A.MASTER_AGMT_GID,
                                                    B.SRCE_EFF_START_TMSP,
                                                    C.PROD_RO_CD || C.POL_SYM_CD || C.POL_NUM POLICY_ID,
                                                    A.POL_EFF_DT_PK_ID,
                                                    C.TERM_WRIT_PREM,
                                                    C.POL_STATUS_CD,
                                                    ROW_NUMBER () OVER (
                                                        PARTITION BY B.AGMT_GID
                                                        ORDER BY
                                                            B.SRCE_EFF_START_TMSP
                                                    ) R_N
                                                FROM
                                                    PRD_PL_DB.APP_SCDI_DM.PL_HOME_POLICY_SNAPSHOT_FACT_VW A,
                                                    PRD_PL_DB.APP_SCDI_DM.PL_AGREEMENT_DIM_VW B,
                                                    PRD_PL_DB.APP_SCDI_DM.PL_CAL_HOME_POLICY_VW C
                                                WHERE
                                                    A.AGMT_GID = B.AGMT_GID
                                                    AND B.POL_PRD_ID = C.POL_PRD_ID
                                                    AND SUBSTR (A.POL_EFF_DT_PK_ID, 1, 4) > '2019'
                                            ) AA
                                        WHERE
                                            AA.R_N = 1
                                    ) D,(
                                        SELECT
                                            POL_ID,
                                            PREV_POL_ID_LATST_PREM
                                        FROM
                                            (
                                                SELECT
                                                    *
                                                FROM
                                                    (
                                                        select
                                                            distinct POLICY_ID POL_ID,
                                                            TERM_WRIT_PREM PREV_POL_ID_LATST_PREM,
                                                            POL_EFF_DT,
                                                            ROW_NUMBER() OVER (
                                                                partition by policy_id
                                                                ORDER BY
                                                                    POL_EFF_DT DESC,
                                                                    POL_PRD_ID DESC
                                                            ) RN
                                                        from
                                                            DSC_PLBI_DB.APP_HOME_PRD.HOME_POLICY_CHAR_REF
                                                    )
                                                WHERE
                                                    RN = 1
                                            )
                                    ) C
                                WHERE
                                    A.POL_PRD_ID = B.POL_PRD_ID
                                    AND A.AGMT_GID = D.AGMT_GID(+)
                                    AND B.PREV_POL_ID = C.POL_ID(+)
                            ) AA
                        WHERE
                            SUBSTR (AA.POL_EFF_DT, 1, 4) > '2019'
                    ) AAA,
                    ULT_PERST_DEV_FCTR BBB,
                    HOME_RENEW_PREM_CHNG_EXPCT CCC
                WHERE
                    AAA.PERSISTENCY_PERIOD = BBB.PERSISTENCY_PERIOD (+)
                    AND AAA.POL_NEW_RNW_CD = BBB.POL_NEW_RENEW_CD(+)
                    AND AAA.POL_PRD_ID = CCC.POL_PRD_ID(+)
            ) AAAA
    );'''  

query2= '''
DROP TABLE DSC_PLBI_DB.APP_HOME_PRD.KEVIN_PETER
'''

# cs.execute(query1)
cs.execute(query2)


cs.close()
ctx.close()
